In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
import pandas as pd
import os
import pickle
import sklearn
import re
from collections import Counter

In [ ]:
def cleanTheFiles(originalPath, savePath):
  dir_list = os.listdir(originalPath)
  for fileName in dir_list:
    f = open(originalPath + "/" + fileName, 'r')
    lines = f.readlines()
    changed_file = []
    for line in lines:
      new_line = line
      #remove all intial semicolons
      new_line = new_line.replace(";", " ", 8)
      #replace the csv required commas into semicolons
      new_line = new_line.replace(",", "; ", 8)
      #remove remaining commas
      new_line = new_line.replace(",", " ")

      new_line = new_line[:-1] + "\n"
      changed_file.append(new_line)
    f.close()

    f = open(savePath + "/" + fileName, 'w')
    f.write("".join(changed_file))


In [ ]:
def getDFforName(fileName):
  print(fileName)
  my_example_df = pd.read_csv("/content/drive/MyDrive/Datathone2023/RiceDatathon2023/data/Bill.com/ocr1/" + fileName, sep = ';')
  #my_example_df = my_example_df.reset_index()

  #display(my_example_df.head(5))

  result_list = list(my_example_df.columns)
  my_example_df.columns = list(range(9))
  #display(my_example_df.head(5))
  my_example_df.loc[len(my_example_df.index)] = result_list
  return my_example_df

In [17]:
savePath = "/content/drive/MyDrive/Datathone2023/RiceDatathon2023/data/Bill.com/ocr1"
originalPath = "/content/drive/MyDrive/Datathone2023/RiceDatathon2023/data/Bill.com/ocr"

cleanTheFiles(originalPath, savePath)


#we know that the first 9 and the last are required, everything else is

KeyboardInterrupt: ignored

In [ ]:
nameToDF = {}
dir_list = os.listdir("/content/drive/MyDrive/Datathone2023/RiceDatathon2023/data/Bill.com/ocr1/")
for fileName in dir_list:
  nameToDF[fileName] = getDFforName(fileName)

In [ ]:

with open("nameToDF.pickle", 'wb') as f:
  # Use the dump() function to write the list to the file
  pickle.dump(nameToDF, f)

In [22]:
with open("/content/nameToDF.pickle", "rb") as file:
    name_to_df = pickle.load(file)

In [38]:
with open("/content/images_to_text_dict.pickle", "rb") as file:
    new_pickle = pickle.load(file)

Use Bag of Words and TF-IDF to convert 




## First, let's convert the bounding box text into string. Each list element below is a receipt. Then, we can concatante them and get bag of words. 

In [163]:
def getNamesToAmounts(nameToDF):
  name_to_amounts = {}

  for name in nameToDF:
    name_to_amounts[name] = []

    receipt_full = " ".join(name_to_df[name].iloc[:, 8].values)
    matches =  re.findall(r'([\$€£¥₹₽₩₱][0-9]+((\.|,)[0-9]+)?)', receipt_full)

    for full_match in matches:
      name_to_amounts[name].append(full_match[0])



In [156]:
def getReceiptCorpusList(nameToDF):
  receipt_list = []
  for name in name_to_df:
    receipt_full = " ".join(name_to_df[name].iloc[:, 8].values)
    

    receipt_full_alpha = re.sub(r'[^a-zA-Z]', ' ', receipt_full).lower()
    receipt_list.append(receipt_full)

In [105]:
from sklearn.feature_extraction.text import CountVectorizer
receipt_list = getReceiptCorpusList()
vectorizer = CountVectorizer(max_features = 500, stop_words = 'english')
vectorizer.fit(receipt_list)

CountVectorizer(max_features=500, stop_words='english')

## We got bag of words and now we can transoform document into features.

In [161]:
amount_list = re.findall(r'([\$€£¥₹₽₩₱][0-9]+((\.|,)[0-9]+)?)', '₩10,000')

In [162]:
amount_list

[('₩10,000', ',000', ',')]

In [154]:
name_to_df['00d0383639350.csv']

,0,1,2,3,4,5,6,7,8
0,2012,1578,2425,1578,2425,1658,2012,1658,(1143655-H)
1,1684,1667,2728,1667,2728,1752,1684,1752,31 31A JALAN HANG TUAH 9
2,1807,1766,2614,1766,2614,1848,1807,1848,TAMAN SKUDAI BARU
3,1938,1867,2475,1867,2475,1940,1938,1940,81300 SKUDAI
4,2081,1957,2350,1957,2350,2028,2081,2028,JOHOR
...,...,...,...,...,...,...,...,...,...
63,2623,4249,2766,4249,2766,4319,2623,4319,0.00
64,1568,4360,2676,4360,2676,4432,1568,4432,# INDICATED THIS TAX CODE BELONG TO SERVICE
65,1576,4426,1802,4426,1802,4495,1576,4495,CHARGES.
66,1643,4646,2690,4646,2690,4718,1643,4718,THANK YOU. PLEASE COME AGAIN !!


In [ ]:
# load up the dataset 
# "20_news_same_line_random_sample.txt" for small dataset 
# "s3://s21risamyersbucket/lab/20_news_same_line.txt" for entire large dataset
corpus = sc.textFile ("20_news_same_line_random_sample.txt")

# each entry in validLines will be a line from the text file
validLines = corpus.filter(lambda x : 'id=' in x)

# now we transform it into a bunch of (docID, text) pairs
#ex of a key 20_newsgroups/comp.graphics/37926
#value is everything starting with 'From'
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('"> ') + 3:x.index(' </doc>')]))

# now we split the text in each (docID, text) pair into a list of words
# after this, we have a data set with (docID, ["word1", "word2", "word3", ...])
# we have a bit of fancy regular expression stuff here to make sure that we do not
# die on some of the documents                       
regex = re.compile('[^a-zA-Z]')  
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

# now get the top 20,000 words... first change (docID, ["word1", "word2", "word3", ...])
# to ("word1", 1) ("word2", 1)...
allWords = keyAndListOfWords.flatMap(lambda x: ((j, 1) for j in x[1]))#get all words counted individually

# now, count all of the words, giving us ("word1", 1433), ("word2", 3423423), etc.
allCounts = allWords.reduceByKey (lambda a, b: a + b)

# and get the top numWords (50 for small dataset, 20K for large dataset) frequent words in a local array
topWords = allCounts.top (numWords, lambda x : x[1])

# and we'll create an RDD that has a bunch of (word, rank) pairs
# start by creating an RDD that has the number 0 up to numWords (50 for small dataset, 20K for large dataset) 
# numWords is the number of words that will be in our dictionary
numWordsRDD = sc.parallelize(range(numWords))

# now, we transform (0), (1), (2), ... to ("mostcommonword", 0) ("nextmostcommon", 1), ...
# the number will be the spot in the dictionary used to tell us where the word is located
refDict = numWordsRDD.map(lambda x:(topWords[x][0],x))#we use topWords[x][0] to get only the first value of th pair